In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('crowds_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'desc_location']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_crowds'

In [12]:
crowds_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['crowds_id'])
              .fillna(value = np.nan, axis = 1))

In [13]:
sql_cols = list(crowds_sql.columns.values)

In [14]:
hash_rows(crowds_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [15]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [16]:
sheet = client.open('Crowds_Combined')

In [17]:
ws = sheet.worksheet('Sheet1')

In [18]:
crowds = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [19]:
#Remove the rows where there timestamp is null because these sheets have extra rows full of nulls
crowds = crowds[crowds['encounter_timestamp'].notnull()]

In [20]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,desc_location,borough
1,5/28/2020 18:43:58,Q-13,50.0,No,Approached the crowd; they ignored the employee,Baseball field,"PATRONS DAMAGED THE FENCE BY CUTTING IT OPEN,,...",Q133A | Hempstead Ballfield,Queens
2,5/28/2020 17:11:11,M-02,60.0,Yes,Approached the crowd; they complied with instr...,Baseball field,NaN,M237 | Corporal John A. Seravalli Playground,Manhattan
3,5/28/2020 15:05:25,M-11,15.0,Yes,Did not approach the crowd; the crowd remains,Adult fitness equipment,NaN,M058-01 | Marcus Garvey Memorial Park-Mt. Morr...,Manhattan
4,5/28/2020 13:18:51,Q-08,2.0,No,Did not approach the crowd; the crowd remains,Handall court,PEP notified,Q220G | Hoover - Manton Playgrounds,Queens
5,5/28/2020 7:41:07,X-14,35.0,No,Did not approach the crowd; the crowd remains,Baseball field,Soccer game going on,X147A-ZN01 | Starlight Park Zone 1,Bronx


In [21]:
yesno = ['in_playground']

In [22]:
yesno_cols(crowds, yesno)

In [23]:
crowds = crowds.merge(site_ref, how = 'left', on = 'desc_location')[sql_cols]

In [24]:
hash_rows(crowds, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [25]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,row_hash
0,5/28/2020 18:43:58,Q-13,50.0,0.0,Approached the crowd; they ignored the employee,Baseball field,"PATRONS DAMAGED THE FENCE BY CUTTING IT OPEN,,...",Q133A,Queens,0f15d0fa3ffa5dc29ec29e6e8cfb0c843acb86c6bdfd38...
1,5/28/2020 17:11:11,M-02,60.0,1.0,Approached the crowd; they complied with instr...,Baseball field,NaN,M237,Manhattan,46a5df98f09cee078c5e356026c99f01cb4b6132012a18...
2,5/28/2020 15:05:25,M-11,15.0,1.0,Did not approach the crowd; the crowd remains,Adult fitness equipment,NaN,M058-01,Manhattan,706127d6bbf3f9af4b9c06cbd8880bd84730df89b06ba5...
3,5/28/2020 13:18:51,Q-08,2.0,0.0,Did not approach the crowd; the crowd remains,Handall court,PEP notified,Q220G,Queens,bc2efbfe5690583dd6b5c34ff03044d62014480bc63af9...
4,5/28/2020 7:41:07,X-14,35.0,0.0,Did not approach the crowd; the crowd remains,Baseball field,Soccer game going on,X147A-ZN01,Bronx,5befea923439092c9cbf7dd84900a869c283e69f588758...


### Find the deltas based on the row hashes

In [26]:
crowds_deltas = (check_deltas(new_df = crowds, old_df = crowds_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [27]:
crowds_deltas.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,dml_verb
0,5/28/2020 18:43:58,Q-13,50.0,0.0,Approached the crowd; they ignored the employee,Baseball field,"PATRONS DAMAGED THE FENCE BY CUTTING IT OPEN,,...",Q133A,Queens,I
1,5/28/2020 17:11:11,M-02,60.0,1.0,Approached the crowd; they complied with instr...,Baseball field,NaN,M237,Manhattan,I
2,5/28/2020 15:05:25,M-11,15.0,1.0,Did not approach the crowd; the crowd remains,Adult fitness equipment,NaN,M058-01,Manhattan,I
3,5/28/2020 13:18:51,Q-08,2.0,0.0,Did not approach the crowd; the crowd remains,Handall court,PEP notified,Q220G,Queens,I
4,5/28/2020 7:41:07,X-14,35.0,0.0,Did not approach the crowd; the crowd remains,Baseball field,Soccer game going on,X147A-ZN01,Bronx,I


In [28]:
crowds_inserts = crowds_deltas[crowds_deltas['dml_verb'] == 'I'][sql_cols]

In [29]:
crowds_inserts.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough
0,5/28/2020 18:43:58,Q-13,50.0,0.0,Approached the crowd; they ignored the employee,Baseball field,"PATRONS DAMAGED THE FENCE BY CUTTING IT OPEN,,...",Q133A,Queens
1,5/28/2020 17:11:11,M-02,60.0,1.0,Approached the crowd; they complied with instr...,Baseball field,NaN,M237,Manhattan
2,5/28/2020 15:05:25,M-11,15.0,1.0,Did not approach the crowd; the crowd remains,Adult fitness equipment,NaN,M058-01,Manhattan
3,5/28/2020 13:18:51,Q-08,2.0,0.0,Did not approach the crowd; the crowd remains,Handall court,PEP notified,Q220G,Queens
4,5/28/2020 7:41:07,X-14,35.0,0.0,Did not approach the crowd; the crowd remains,Baseball field,Soccer game going on,X147A-ZN01,Bronx


In [30]:
crowds_inserts.to_sql('tbl_dpr_crowds', engine, index = False, if_exists = 'append')

In [31]:
crowds_updates = crowds_deltas[crowds_deltas['dml_verb'] == 'U'][sql_cols]

In [32]:
crowds_updates.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough


In [33]:
sql_update(crowds_updates, 'tbl_dpr_crowds', engine, ['encounter_timestamp', 'site_id'])